In [0]:
import sys
workspace = dbutils.widgets.get('workspace')
sys.path.insert(0, workspace)

from libs.ingestao_silver import ingestao_silver

from pyspark.sql.functions import col, cast, trim, upper, when, to_date

In [0]:
catalog = 'projeto_cnpj'
table = 'socios'
df = spark.table(f"{catalog}.bronze.{table}")

df= (df
    .withColumn(
        'identificador_socio', 
        when(trim(col('identificador_socio')) == '1', 'Pessoa Juridica')
        .when(trim(col('identificador_socio')) == '2', 'Pessoa Fisica')
        .when(trim(col('identificador_socio')) == '3', 'Estrangeiro')
        .otherwise('Outros')
    )
    .withColumn('nome_socio', upper(trim(col('nome_socio'))))
    .withColumn('qualificacao_socio', col('qualificacao_socio').cast('int'))
    .withColumn('data_entrada', to_date(col('data_entrada'), 'yyyyMMdd'))
    .withColumn('qualificacao_representante', col('qualificacao_representante').cast('int'))
    .withColumn('faixa_etaria', col('faixa_etaria').cast('int'))
    .withColumn('data_ingestao', col('data_ingestao').cast('date'))
)


In [0]:
ingestao_silver(
    spark = spark,
    df = df,
    catalog = catalog,
    table = table
)